# 05-01 : Basic Function Test

In [1]:
import os

In [2]:
import logging

# configure log format
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s [%(name)s] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# get the logger
logger = logging.getLogger(__name__)

## 1. Configuration

In [3]:
MLRUN_API = "http://ac7245a542f7a4979b638e13f202683b-c3d8b5ba86e0bb61.elb.af-south-1.amazonaws.com"
#MLRUN_API = "http://dragon:30070"

os.environ["MLRUN_DBPATH"] = MLRUN_API
print(f"MLRUN_DBPATH: {os.environ['MLRUN_DBPATH']}")

MLRUN_DBPATH: http://ac7245a542f7a4979b638e13f202683b-c3d8b5ba86e0bb61.elb.af-south-1.amazonaws.com


In [4]:
import mlrun

## 1. Set Project

In [5]:
project_name = 'johannes-was-here'

project = mlrun.get_or_create_project(
    name=project_name,
    context='./',) 

project.save()

> 2025-06-26 00:16:51,186 [info] loaded project johannes-was-here from MLRun DB


## 2. Create Function

### 2.1 Create Function Code File

In [6]:
%%writefile hello_world.py

import mlrun

# get the logger
import logging
logger = logging.getLogger(__name__)

def handler(context: mlrun.MLClientCtx = None):
    logger.info("Hello world")

    context.log_result("hello", "world")

Writing hello_world.py


### 2.2 Create Project Function

In [21]:
hello_world = mlrun.code_to_function(
    name='hello-world',
    project=project_name,
    filename='hello_world.py',
    handler='handler',
    kind='job',
    description='Hello world function',
)
hello_world.save()
hello_world.deploy()

True

## 3. Run Function

In [ ]:
# 1.7.2
# run = hello_world.run(function=hello_world,
#                       handler='handler')

# 1.0.5
run = hello_world.run(name="hello_world",
                      handler='handler')

> 2025-06-26 00:25:36,640 [info] starting run hello_world uid=f944e18bc8d64d95ab74d5883499c87c DB=http://ac7245a542f7a4979b638e13f202683b-c3d8b5ba86e0bb61.elb.af-south-1.amazonaws.com
> 2025-06-26 00:25:36,864 [info] Job is running in the background, pod: hello-world-hrwsr
> 2025-06-25 22:25:44,147 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
johannes-was-here,...99c87c,0,Jun 25 22:25:44,completed,hello_world,kind=jobowner=fouldsmlrun/client_version=1.0.5host=hello-world-hrwsr,,,hello=world,


> 2025-06-26 00:25:46,311 [info] run executed, status=completed


In [26]:
# get the run results
logger.info(f"Run ID: {run.metadata.uid}")
logger.info(f"Run state: {run.status.state}")
logger.info(f"Run results: {run.status.results}")

2025-06-26 00:28:34 INFO [__main__] Run ID: f944e18bc8d64d95ab74d5883499c87c
2025-06-26 00:28:34 INFO [__main__] Run state: completed
2025-06-26 00:28:34 INFO [__main__] Run results: {'hello': 'world'}
